In [1]:
import os
import re
import json

from urllib.parse import urlparse

In [2]:
count = 0
for dir in os.listdir("packages/packages"):
    count += len(os.listdir(f"packages/packages/{dir}"))
print(count)

6135


In [3]:
search_keys = set(["security", "fingerprinting", "privacy", "sha"])

In [5]:
matched = []
pkg_path = "packages/packages"

for dir in os.listdir(pkg_path):
    for file in os.listdir(f"{pkg_path}/{dir}"):
        fp = open(f"{pkg_path}/{dir}/{file}")
        json_obj = json.load(fp)
        keywords = set(json_obj["keywords"])
        
        if keywords.intersection(search_keys):
            matched.append(
                {
                    "name": json_obj["name"],
                    "filename": json_obj["filename"]
                }
            )

In [6]:
matched

[{'name': 'angular-recaptcha', 'filename': 'angular-recaptcha.min.js'},
 {'name': 'asmCrypto', 'filename': 'asmcrypto.all.es5.min.js'},
 {'name': 'crypto-js', 'filename': 'crypto-js.min.js'},
 {'name': 'dompurify', 'filename': 'purify.min.js'},
 {'name': 'fingerprintjs2', 'filename': 'fingerprint2.min.js'},
 {'name': 'jquery.complexify.js', 'filename': 'jquery.complexify.min.js'},
 {'name': 'jquery.hashcash.io', 'filename': 'jquery.hashcash.io.min.js'},
 {'name': 'jquery.socialshareprivacy',
  'filename': 'jquery.socialshareprivacy.min.js'},
 {'name': 'js-sha1', 'filename': 'sha1.min.js'},
 {'name': 'js-sha256', 'filename': 'sha256.min.js'},
 {'name': 'js-sha512', 'filename': 'sha512.min.js'},
 {'name': 'js-xss', 'filename': 'xss.min.js'},
 {'name': 'oauth-io', 'filename': 'oauth.min.js'},
 {'name': 'zxcvbn', 'filename': 'zxcvbn.js'}]

In [7]:
from datasets import load_dataset

In [8]:
ds = load_dataset("big-banyan-tree/BBT_CommonCrawl_2024", data_dir="script_extraction", split="train")

Resolving data files:   0%|          | 0/58 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/57 [00:00<?, ?it/s]

In [9]:
sds = ds.select(range(len(ds) // 5))

In [10]:
del ds

In [11]:
len(sds)

4631344

In [12]:
regex = re.compile(r"/?(?:js\.)?([^/?]+\.js)")

In [13]:
def attrs_to_libs(row):
    """Parse list of src attrs to have JS libs ONLY."""
    splits = row["script_src_attrs"].split('|')
    for i, s in enumerate(splits):
        m = regex.search(s)
        if m:
            splits[i] = m.group(1)
        else:
            splits[i] = None
            continue
        splits[i] = re.sub(r"\.min\.", '.', splits[i])
            
    return {"js_lib": splits}

In [14]:
def extract_domain(row):
    parsed_url = urlparse(row["host"])
    return {"domain": parsed_url.netloc}

In [15]:

    # for i, attrs in enumerate(src_attrs):
    #     splits = attrs.split('|')
    #     for j, s in enumerate(splits):
    #         m = regex.search(s)
    #         if m:
    #             splits[j] = m.group(1)
    #         else:
    #             splits[j] = None
    #             continue
    #         splits[j] = re.sub(r"\.min\.", '.', splits[j])
            
    # return splits

In [16]:
new_ds = sds.map(extract_domain, num_proc=4)

Map (num_proc=4):   0%|          | 0/4631344 [00:00<?, ? examples/s]

In [17]:
new_ds = new_ds.map(attrs_to_libs, num_proc=4)

Map (num_proc=4):   0%|          | 0/4631344 [00:00<?, ? examples/s]

In [18]:
# import pickle

In [19]:
# with open("new_ds.pkl", 'wb') as f:
#     pickle.dump(new_ds, f)

In [20]:
# with open("new_ds.pkl", 'rb') as f:
#     tds = pickle.load(f)

In [21]:
df = new_ds.to_pandas()

In [23]:
del new_ds

In [24]:
del sds

In [22]:
df.head()

,ip,host,server,script_src_attrs,year,domain,js_lib
0,101.32.10.132,http://www.buyjia.net/company-guangdongzhongzh...,nginx/1.20.1,/js/jquery-3.1.1.min.js|/js/bootstrap.min.js,2024,www.buyjia.net,"[jquery-3.1.1.js, bootstrap.js]"
1,201.49.2.61,https://simoesfilho.ba.gov.br/simoes-filho-cam...,Apache,//www.googletagmanager.com/gtag/js?id=g-4cg97x...,2024,simoesfilho.ba.gov.br,"[None, jquery.js, jquery-migrate.js, frontend-..."
2,194.163.38.164,https://stalionelanka.com/management/,LiteSpeed,https://stalionelanka.com/wp-includes/js/jquer...,2024,stalionelanka.com,"[jquery.js, jquery-migrate.js, layerslider.uti..."
3,54.204.238.15,https://app.vagrantup.com/boxes/search?archite...,Cowboy,/assets/application-d926c034f15c2fb3f06276d6e7...,2024,app.vagrantup.com,[application-d926c034f15c2fb3f06276d6e7e132e6b...
4,185.220.174.17,https://sosstraling.nl/film-over-wally/,Apache/2,https://sosstraling.nl/wp-includes/js/jquery/j...,2024,sosstraling.nl,"[jquery.js, jquery-migrate.js, avia-js.js, avi..."


In [25]:
df = df.explode("js_lib")

In [26]:
df.head()

,ip,host,server,script_src_attrs,year,domain,js_lib
0,101.32.10.132,http://www.buyjia.net/company-guangdongzhongzh...,nginx/1.20.1,/js/jquery-3.1.1.min.js|/js/bootstrap.min.js,2024,www.buyjia.net,jquery-3.1.1.js
0,101.32.10.132,http://www.buyjia.net/company-guangdongzhongzh...,nginx/1.20.1,/js/jquery-3.1.1.min.js|/js/bootstrap.min.js,2024,www.buyjia.net,bootstrap.js
1,201.49.2.61,https://simoesfilho.ba.gov.br/simoes-filho-cam...,Apache,//www.googletagmanager.com/gtag/js?id=g-4cg97x...,2024,simoesfilho.ba.gov.br,None
1,201.49.2.61,https://simoesfilho.ba.gov.br/simoes-filho-cam...,Apache,//www.googletagmanager.com/gtag/js?id=g-4cg97x...,2024,simoesfilho.ba.gov.br,jquery.js
1,201.49.2.61,https://simoesfilho.ba.gov.br/simoes-filho-cam...,Apache,//www.googletagmanager.com/gtag/js?id=g-4cg97x...,2024,simoesfilho.ba.gov.br,jquery-migrate.js


In [28]:
df.drop_duplicates(subset=["domain", "js_lib"], inplace=True)

In [29]:
df.head()

,ip,host,server,script_src_attrs,year,domain,js_lib
0,101.32.10.132,http://www.buyjia.net/company-guangdongzhongzh...,nginx/1.20.1,/js/jquery-3.1.1.min.js|/js/bootstrap.min.js,2024,www.buyjia.net,jquery-3.1.1.js
0,101.32.10.132,http://www.buyjia.net/company-guangdongzhongzh...,nginx/1.20.1,/js/jquery-3.1.1.min.js|/js/bootstrap.min.js,2024,www.buyjia.net,bootstrap.js
1,201.49.2.61,https://simoesfilho.ba.gov.br/simoes-filho-cam...,Apache,//www.googletagmanager.com/gtag/js?id=g-4cg97x...,2024,simoesfilho.ba.gov.br,None
1,201.49.2.61,https://simoesfilho.ba.gov.br/simoes-filho-cam...,Apache,//www.googletagmanager.com/gtag/js?id=g-4cg97x...,2024,simoesfilho.ba.gov.br,jquery.js
1,201.49.2.61,https://simoesfilho.ba.gov.br/simoes-filho-cam...,Apache,//www.googletagmanager.com/gtag/js?id=g-4cg97x...,2024,simoesfilho.ba.gov.br,jquery-migrate.js


In [31]:
df.iloc[2]["js_lib"]

In [33]:
filenames = [re.sub(r"\.min\.", '.', item['filename']) for item in matched]

In [42]:
filenames.remove("crypto-js.js")

In [43]:
filenames

['angular-recaptcha.js',
 'asmcrypto.all.es5.js',
 'purify.js',
 'fingerprint2.js',
 'jquery.complexify.js',
 'jquery.hashcash.io.js',
 'jquery.socialshareprivacy.js',
 'sha1.js',
 'sha256.js',
 'sha512.js',
 'xss.js',
 'oauth.js',
 'zxcvbn.js']

In [35]:
df.columns

Index(['ip', 'host', 'server', 'script_src_attrs', 'year', 'domain', 'js_lib'], dtype='object')

In [44]:
filtered_df = df[df['js_lib'].isin(filenames)][['domain', 'js_lib']]

In [45]:
filtered_df.head(n=50)

,domain,js_lib
72,www.marshallcountydaily.com,fingerprint2.js
702,www.tekforums.net,sha1.js
883,openi.pcl.ac.cn,purify.js
1134,bucurestiulmeudrag.ro,fingerprint2.js
2045,www.bender.de,purify.js
2303,www.lestoilesdularge.com,fingerprint2.js
2516,www.index.hr,fingerprint2.js
2912,dreamcast.org.ru,sha1.js
2980,www.gemeinde-goeriach.at,jquery.socialshareprivacy.js
3152,forum.grsu.by,sha1.js


In [46]:
len(filtered_df)

5999

In [48]:
filtered_df.to_csv("security_domains.csv")